# ============================================
# LangGraph – Graph Logic Fundamentals (Nodes, Edges, START/END, Conditionals)
# ============================================
**Author:** Dr. Dasha Trofimova

### Goals
- Understand **nodes** (pure Python functions) and **state flow**
- Use **START** (entry point) and **END** (termination)
- Wire **edges** and **conditional edges** for routing
- Visualize graphs with Graphviz & trace execution

---


In [ ]:
!pip install -q langgraph graphviz

## 1) Minimal linear graph: START → A → B → END

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class S(TypedDict):
    x: int
    log: str

def A(s: S) -> S:
    return {"x": s["x"] + 1, "log": s["log"] + "A "}

def B(s: S) -> S:
    return {"x": s["x"] * 2, "log": s["log"] + "B "}

g = StateGraph(S)
g.add_node("A", A); g.add_node("B", B)
g.add_edge("A", "B"); g.add_edge("B", END)
g.set_entry_point("A")
app = g.compile()
print(app.invoke({"x": 3, "log":"START "}))

## 2) Conditional edge: route based on state

We’ll branch to `POS` if `x >= 0` else to `NEG`.


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class S2(TypedDict):
    x: int
    route: str

def decide(s: S2) -> S2:
    return {"x": s["x"], "route": "POS" if s["x"] >= 0 else "NEG"}

def POS(s: S2) -> S2:
    return {"x": s["x"], "route": s["route"] + " → POS"}

def NEG(s: S2) -> S2:
    return {"x": s["x"], "route": s["route"] + " → NEG"}

def router(s: S2):
    return s["route"]

g2 = StateGraph(S2)
g2.add_node("decide", decide)
g2.add_node("POS", POS); g2.add_node("NEG", NEG)
g2.add_conditional_edges("decide", router, {"POS":"POS", "NEG":"NEG"})
g2.add_edge("POS", END); g2.add_edge("NEG", END)
g2.set_entry_point("decide")

print(g2.compile().invoke({"x": 5, "route": ""}))
print(g2.compile().invoke({"x": -2, "route": ""}))

## 3) Loop (feedback edge)

We’ll decrement `x` until it reaches 0.


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class L(TypedDict):
    x: int
    steps: int

def dec(l: L) -> L:
    return {"x": max(0, l["x"]-1), "steps": l["steps"]+1}

def should_continue(l: L):
    return l["x"] > 0

g3 = StateGraph(L)
g3.add_node("dec", dec)
g3.add_conditional_edges("dec", should_continue, {True: "dec", False: END})
g3.set_entry_point("dec")
print(g3.compile().invoke({"x": 4, "steps": 0}))

## 4) Visualize with Graphviz

In [ ]:
from graphviz import Digraph
dot = Digraph(format="png"); dot.attr(rankdir="LR")
dot.node("A","A()", shape="box"); dot.node("B","B()", shape="box"); dot.node("END","END", shape="doublecircle")
dot.edge("A","B"); dot.edge("B","END"); dot

## 5) Execution tracing — see which nodes executed

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Callable, Dict

class T(TypedDict):
    q: str
    path: str

def trace(name: str, desc: str):
    def deco(fn):
        def wrap(s: T) -> T:
            print(f"▶ {name}: {desc}")
            out = fn(s)
            out["path"] = s.get("path","") + f"{name} → "
            return out
        wrap.__doc__ = desc
        return wrap
    return deco

@trace("decide", "Router: sends to POS if q contains 'ok', else NEG")
def decide(s: T) -> T:
    return {"q": s["q"], "path": s.get("path",""), **({"route":"POS"} if "ok" in s["q"] else {"route":"NEG"})}

@trace("POS", "Positive node")
def POS(s: T) -> T: return {"q": s["q"], "path": s["path"]}

@trace("NEG", "Negative node")
def NEG(s: T) -> T: return {"q": s["q"], "path": s["path"]}

def router(s: Dict): return s["route"]

g = StateGraph(T)
g.add_node("decide", decide); g.add_node("POS", POS); g.add_node("NEG", NEG)
g.add_conditional_edges("decide", router, {"POS":"POS","NEG":"NEG"})
g.add_edge("POS", END); g.add_edge("NEG", END)
g.set_entry_point("decide")
res = g.compile().invoke({"q":"all ok", "path":""})
print("PATH:", res["path"] + "END")

### ✅ Takeaways
- **Node:** a pure function `State -> State`
- **Edges:** connect nodes in sequence; **conditional edges** pick a branch
- **START/END:** set entry point with `set_entry_point`, finish by connecting to `END`
- **Trace & visualize** to make flows teachable

---

### 🎯 Quick Card Quiz — Graph Logic
- **White** = Node
- **Brown** = Conditional Edge
- **Green** = END

1) What connects one node’s output to the next node?
2) What construct chooses the next node at runtime? 
3) What indicates that no further nodes run?
